In [1]:
import findspark
findspark.init()

import pyspark
import pyspark.sql
from pyspark.sql import *
import os.path
from pyspark.sql.functions import desc

from pyspark.sql import dataframe
from pyspark.sql import functions as F
from pyspark.sql.functions import isnan, when, count, col

from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

from pyspark.sql.types import *

import wptools
import pandas as pd
import sys, os

_stderr = sys.stderr
null = open(os.devnull,'wb')

In [2]:
DATA_DIR = '../data/data_processed/' 

In [23]:
articles = pd.read_csv(DATA_DIR + 'data_processed.csv', index_col=0)
articles.index.names = ['id']

In [4]:
articles.count()

title             17459
categories        17459
article_lenght    17459
views             17459
refs_count        17459
link_count        17459
death             17459
locations         17289
end_date_clean    16848
ongoing           17459
dtype: int64

In [5]:
articles = articles.reset_index()

In [17]:
def get_wiki_military_conflict_deaths(row):
    idPage = row['id']
    page = wptools.page(pageid=idPage)
    
    try:
        page.get_parse()
    except:
        return 0
    
    deaths = None
    
    try: 
        page.get_wikidata()
        deaths = page.data['wikidata']['number of deaths (P1120)']['amount']
        return deaths
    except:
        deaths = None
        
    try:
        deaths = page.data['infobox']['casualties3']
        return deaths
    except:
        deaths = None
        
    
    deaths2 = None
    
    try:
        deaths = page.data['infobox']['casualties1']
    except:
        return None
        
    try:
        deaths2 = page.data['infobox']['casualties2']
        deaths  = deaths + " |caus2| " + deaths2
    except:
        deaths2 = None
        
    if deaths:
        return deaths
    else:
        return row['death']

In [18]:
new_articles=articles[(articles.end_date_clean >= 1995) & (articles.categories.str.contains("conflict"))]
new_articles

,id,title,categories,article_lenght,views,refs_count,link_count,death,locations,end_date_clean,ongoing
39,16760,Kosovo War,military conflict,197476,2368366,274,671,NaN,Kosovo Q1231,1999.0,False
44,22738,Operation Enduring Freedom,military conflict,52281,1248100,60,228,NaN,Afghanistan Q889,2014.0,False
57,30770,The Troubles,military conflict,143560,4660829,214,578,NaN,Northern Ireland Q26,1998.0,False
83,43063,Operation Anaconda,military conflict,53930,428455,34,205,NaN,Shah- i- Kot Valley Q16205112,2002.0,False
198,71717,Second Intifada,military conflict,193420,680777,261,784,NaN,Palestinian Authority|Israel,2005.0,False
269,140792,First Chechen War,military conflict,63641,985456,66,308,NaN,Chechen Republic Q5187|Republic of Ingushe...,1996.0,False
408,171392,Second Chechen War,military conflict,134225,1385837,167,634,NaN,Chechen Republic Q5187,2009.0,False
425,182821,Siege of Sarajevo,military conflict,78135,1060559,85,365,NaN,Sarajevo Q11194,1996.0,False
463,201936,2003 invasion of Iraq,military conflict,250155,2953828,290,886,NaN,Iraq Q796,2003.0,False
464,202304,Protests against the Iraq War,civil conflict,85504,134088,111,317,NaN,Global,2002.0,False


In [20]:
%%capture
new_articles['death'] = new_articles.apply(get_wiki_military_conflict_deaths,axis=1)

In [27]:
new_articles = new_articles.set_index('id')

In [30]:
old_articles = articles

In [40]:
articles.update(new_articles)

In [54]:
articles = articles.reset_index()

In [58]:
articles['id'] = articles.id.astype(int)

In [60]:
articles = articles.set_index('id')

In [67]:
articles = articles.drop(['index'],axis=1)

ok

In [69]:
articles.to_csv(DATA_DIR + 'data_processed_before_death_cleaning.csv')

In [70]:
articles

,title,categories,article_lenght,views,refs_count,link_count,death,locations,end_date_clean,ongoing
id,,,,,,,,,,
16760,Kosovo War,military conflict,197476.0,2368366.0,274.0,671.0,[[File:UCK KLA.png|25px|alt=Logo of the Kosovo...,Kosovo Q1231,1999.0,False
22738,Operation Enduring Freedom,military conflict,52281.0,1248100.0,60.0,228.0,"{{flagicon|Afghanistan}} '''45,000+''' killed<...",Afghanistan Q889,2014.0,False
30770,The Troubles,military conflict,143560.0,4660829.0,214.0,578.0,UDA: 91<br />UVF: 62<br />RHC: 4<br />LVF: 3<b...,Northern Ireland Q26,1998.0,False
43063,Operation Anaconda,military conflict,53930.0,428455.0,34.0,205.0,Unknown number of Afghan fighters killed {{Fla...,Shah- i- Kot Valley Q16205112,2002.0,False
71717,Second Intifada,military conflict,193420.0,680777.0,261.0,784.0,55 foreign citizens total:<br />- 45 foreign c...,Palestinian Authority|Israel,2005.0,False
140792,First Chechen War,military conflict,63641.0,985456.0,66.0,308.0,"30,000–40,000 civilians killed ([[Russian Fede...",Chechen Republic Q5187|Republic of Ingushe...,1996.0,False
171392,Second Chechen War,military conflict,134225.0,1385837.0,167.0,634.0,'''[[Casualties of the Second Chechen War|Civi...,Chechen Republic Q5187,2009.0,False
182821,Siege of Sarajevo,military conflict,78135.0,1060559.0,85.0,365.0,"5,434 civilians killed",Sarajevo Q11194,1996.0,False
201936,2003 invasion of Iraq,military conflict,250155.0,2953828.0,290.0,886.0,Estimated [[Casualties of the Iraq War#Iraqi i...,Iraq Q796,2003.0,False
